# Explore TPM data matrix

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord

from override import GENE_NAME_OVERRIDE, GENE_PRODUCT_OVERRIDE



In [2]:
# feature list indices
LEFT_IDX = 0
RIGHT_IDX = 1
STRAND_IDX = 2
LOCUS_IDX = 3
GENE_IDX = 4
TYPE_IDX = 5

In [3]:
# load TPM data
df = pd.read_csv('data/extract_TPM_counts.tsv',sep='\t').fillna('')

# load mapping from sample to condition
with open('data/sample2condition.txt','r') as f:
    sample2condition = dict(x.strip().split() for x in f.readlines())

# load sample to include file
samples_provided = False
if samples_provided:
    with open('config/samples_to_include.txt','r') as f:
        samples = list(x.strip() for x in f.readlines())
# if none provided, just use all the samples from the sample2condition dict
else: 
    samples = list(sample2condition.keys())
    
# load the conditions to include file
with open('config/conditions_to_include.txt','r') as f:
    conditions = list(x.strip() for x in f.readlines())


In [4]:
df.head()

,locus_tag,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,gene_len,...,5GB1_pA9_red_tpm,5GB1_pA9_yellow_tpm,5GB1C-5G-La-BR1_tpm,5GB1C-5G-La-BR2_tpm,5GB1C-5G-N-BR1_tpm,5GB1C-5G-N-BR2_tpm,5GB1C-JG15-La-BR1_tpm,5GB1C-JG15-La-BR2_tpm,5GB1C-JG15-N-BR1_tpm,5GB1C-JG15-N-BR2_tpm
0,EQU24_RS00005,chromosomal replication initiator protein DnaA,CDS,dnaA,NZ_CP035467.1,0,1317,Derived by automated computational analysis us...,MSALWNNCLAKLENEISSSEFSTWIRPLQAIETDGQIKLLAPNRFV...,1318,...,38.557373,38.810668,37.444214,40.246006,40.100118,33.432274,39.880174,38.355431,30.247582,41.248441
1,EQU24_RS00010,DNA polymerase III subunit beta,CDS,,NZ_CP035467.1,1502,2603,Derived by automated computational analysis us...,MKYIINREQLLVPLQQIVSVIEKRQTMPILSNVLMVFRENTLVMTG...,1102,...,52.552767,52.461746,42.676553,49.210083,46.798476,48.142385,45.465136,46.498139,37.152951,52.902410
2,EQU24_RS00015,DNA replication/repair protein RecF,CDS,recF,NZ_CP035467.1,3060,4140,Derived by automated computational analysis us...,MSLQKLDIFNVRNIRQASLQPSPGLNLIYGANASGKSSVLEAIFIL...,1081,...,31.350991,34.914128,21.479309,24.204682,22.171104,22.006566,22.658157,22.753325,19.407103,29.834124
3,EQU24_RS00020,DNA topoisomerase (ATP-hydrolyzing) subunit B,CDS,gyrB,NZ_CP035467.1,4185,6600,Derived by automated computational analysis us...,MSENIKQYDSTNIQVLKGLDAVRKRPGMYIGDTDDGTGLHHMVFEV...,2416,...,74.848501,80.850761,54.959319,64.911376,59.653059,64.648318,69.119079,65.643179,57.590223,68.306759
4,EQU24_RS00025,hypothetical protein,CDS,,NZ_CP035467.1,6825,7062,Derived by automated computational analysis us...,VKTTKYFLTTRMRPDREIIKDEWIQYVVRFPENEHIQFDGRIRRWA...,238,...,50.324948,49.349547,34.539657,36.521074,37.789611,39.358066,38.992158,35.870964,41.462392,40.227192


In [5]:
# for the loaded TPM matrix, which column contains the unique gene ids?
LOCUS_ID_COL = 'locus_tag'
# use this column to get a full list of all genes for which expression was measured
LOCI = list(df[LOCUS_ID_COL].values)

In [6]:
# pivot dataframe to make experiments the rows and genes the columns
exp_df = df.set_index('locus_tag')[samples].T.reset_index().rename(columns={'index':'sample'})
# add experimental condition column to dataframe
exp_df['exp_condition'] = exp_df['sample'].apply(lambda x: sample2condition[x])
# filter to only conditions that are requested
print(exp_df.shape)
exp_df = exp_df[exp_df['exp_condition'].isin(conditions)]
print(exp_df.shape)

exp_df.head()

(98, 4215)
(80, 4215)


locus_tag,sample,EQU24_RS00005,EQU24_RS00010,EQU24_RS00015,EQU24_RS00020,EQU24_RS00025,EQU24_RS00030,EQU24_RS00035,EQU24_RS00040,EQU24_RS00045,...,EQU24_RS22115,EQU24_RS22120,EQU24_RS22125,EQU24_RS22130,EQU24_RS22135,EQU24_RS22140,EQU24_RS22145,EQU24_RS22150,EQU24_RS22155,exp_condition
2,5GB1_FM03_TR1_QC_tpm,48.864921,51.315629,33.906257,64.584281,74.801354,13.090237,50.320911,20.811187,21.910217,...,56.948463,52.002311,60.628466,23.015801,104.488835,379.686212,500.129230,1191.565348,796.964423,uMax
3,5GB1_FM03_TR2_QC_tpm,52.197450,54.947425,29.979783,72.717824,62.709260,24.874673,71.255094,19.773190,19.421485,...,50.243190,41.376658,66.581832,54.669612,103.864151,401.742811,495.262554,1232.784967,724.291958,uMax
4,5GB1_FM11_TR1_QC_tpm,25.751902,37.216017,21.716802,53.032824,46.348014,42.426259,58.772307,9.667881,8.212313,...,25.635202,29.059082,59.545627,34.189659,78.264069,208.094267,260.011454,666.589153,461.697526,lowO2_fast_growth
5,5GB1_FM11_TR2_QC_tpm,34.928170,40.094206,27.871558,55.595529,57.992363,26.299890,63.611578,16.724890,11.858533,...,30.765310,37.468337,64.670046,32.369096,59.685923,210.820167,331.355028,860.248081,503.720248,lowO2_fast_growth
6,5GB1_FM12_TR1_tpm,28.322788,27.387700,16.570863,36.795081,27.432559,22.380161,75.688839,17.920465,11.197050,...,16.227307,21.206868,17.075168,18.146243,58.833576,204.760175,300.766500,828.322141,544.969944,lowCH4


In [15]:
def load_override_dict(filename):
    with open(filename, 'r') as f:
        lines = [x.strip().split('\t') for x in f.readlines()]
        override_dict = dict(lines)
        
    return override_dict

gene_ov = load_override_dict("data/gene_name_override.txt")
prod_ov = load_override_dict("data/gene_product_override.txt")

In [18]:
prod_ov

{'EQU24_RS05925': 'likely chaperone for smmo'}

In [7]:
def get_average_tpm_by_condition(df_orig,
                                 loci,
                                 add_pseudocount=True,
                                 pseudocount = 0.01
                                ):
    '''
    Given a matrix of samples by genes, group the samples by their experimental condition.
    Then for each gene, calculated the average and standard deviation
    '''
    
    # start with a fresh copy of the dataframe
    df = df_orig.copy()
    
    # add a pseudocount to all genes' TPMs
    if add_pseudocount:
        df[loci] = df[loci] + pseudocount 
    
    df_means = df[['exp_condition']+loci]\
                    .groupby('exp_condition')\
                    .mean()\
                    .reset_index()
    
    return df_means
    
def get_top_n_perc_by_condition(df, loci, n):
    '''
    Given a dataframe of experimental conditions and the average TPM of each gene,
    loop through each condition and rank the genes in descending TPM order. Keep of list
    of genes that make the top n% in every condition
    '''
    num_loci = len(loci)
    # keep track of a list of top loci for each condition
    # key = exp_condition, value = (gene,tpm)
    top_n_loci = {}
    
    # for each experimental condition row
    for i,row in df.iterrows():
        exp_cond = row.exp_condition
        #print(exp_cond)
        # Sort all genes in the row by their tpm counts
        tpms = sorted(
                list(zip(loci, row[loci].values)),  # zip locus and TPM
                key=lambda x: x[1],                # sort by TPM
                reverse=True                       # descending TPM order
        )
    
        # keep the top n% of the list
        top_n_loci[exp_cond] = tpms[:int(0.01*n*num_loci)]
        
    # keep only the genes that were in all lists via set intersection
    top_n_loci_only = [[y[0] for y in top_n_loci[x]] for x in top_n_loci] # drop tpm
    top_n_sets = [set(x) for x in top_n_loci_only]                        # convert to sets
    top_n_loci_all_conds = set.intersection(*top_n_sets)  # get intersection of all sets

    return top_n_loci_all_conds#, top_n_loci, top_n_sets

def get_override_gene(locus_tag,cur_gene):
    '''
    Given a locus tag, return an overridden gene
    '''
    return GENE_NAME_OVERRIDE[locus_tag] if locus_tag in GENE_NAME_OVERRIDE else cur_gene


def get_features_from_genbank(gb_file,
                              override=False):
    '''
    Given a genbank file, parse out all of it's features into a 5-tuple 
    of (start_coord, end_coord,locus_tag,gene_symbol,type).
    '''
    # Use BioPython genbank parser
    seq_record = SeqIO.parse(gb_file, "genbank").__next__()
    
    feat_list = []
    # Loop over the genome file, get the CDS features on each of the strands
    for feature in seq_record.features:
        if 'locus_tag' in feature.qualifiers:
            # get  locus tag 
            lt = feature.qualifiers['locus_tag'][0]
            # get the gene symbol if available, otherwise leave blank
            g = "" if 'gene' not in feature.qualifiers else feature.qualifiers['gene'][0]

            # if there's a customized override file to rename certain genes, apply that here
            if override:
                g = get_override_gene(lt,g)
            
            feat_list.append((feature.location.start.position,
                             feature.location.end.position,
                             feature.strand,
                             lt,
                             g,
                             feature.type))
            
    return feat_list
    
    

def flag_potential_operon_loci(pos_feats, neg_feats, min_dist):
    '''
    Given a all features on the pos and neg strands, flag which ones 
    may be in an operon at the given min_dist between loci on the same strand.
    
    Return a set of flagged loci.
    '''
    
    # collect locus tags that are potential operon candidates
    operon_candidate_loci = []
    
    # +-----------------+
    # | NEGATIVE STRAND |
    # +-----------------+
    # if we're in the negative list, check the next feature to the right
    for i,(left_coord,right_coord,strand,locus,gene,typee) in enumerate(neg_feats):
        # if this isn't the last gene (aka no rightward operon),
        # check the min_distance window for other annotations
        if i < len(neg_feats) -1:
            # get the FOLLOWING feature (because on -1 strand)
            upstream_loc = neg_feats[i+1]

            # if the left side of the upstream gene is within min_distance, 
            # collect it
            if upstream_loc[LEFT_IDX] < (right_coord + min_dist):
                operon_candidate_loci.append(neg_feats[i][LOCUS_IDX])
    
    # +-----------------+
    # | POSITIVE STRAND |
    # +-----------------+
    # if we're in the positive list, check the next feature to the left
    for i,(left_coord,right_coord,strand,locus,gene,typee) in enumerate(pos_feats):
        # if this isn't the first gene (aka no leftward operon),
        # check the min_distance window for other annotations
        if i !=0:
            # get the PREVIOUS feature (because on +1 strand)
            upstream_loc = pos_feats[i-1]

            # if the right side of the upstream gene is within min_distance, 
            # collect it
            if upstream_loc[RIGHT_IDX] > (left_coord - min_dist):
                operon_candidate_loci.append(pos_feats[i][LOCUS_IDX])
    
    return set(operon_candidate_loci)


def write_loci_to_file(loci,loci_op_filt, outfile):
    '''
    Given a list of locus tags, and a list of locus_tags possbily in an operon,
    write them to a simple tab delimited file with TRUE next to possible operons
    '''
    
    with open(outfile,'w') as f:
        # set header to be "locus_tag" \tab "op?" \newline
        f.write('locus_tag\top?\n')
        
        for loc in loci:
            op = True if loc in loci_op_filt else False
            f.write(f'{loc}\t{op}\n')
    

In [8]:
df_means = get_average_tpm_by_condition(exp_df,LOCI)
df_means.head()

locus_tag,exp_condition,EQU24_RS00005,EQU24_RS00010,EQU24_RS00015,EQU24_RS00020,EQU24_RS00025,EQU24_RS00030,EQU24_RS00035,EQU24_RS00040,EQU24_RS00045,...,EQU24_RS22110,EQU24_RS22115,EQU24_RS22120,EQU24_RS22125,EQU24_RS22130,EQU24_RS22135,EQU24_RS22140,EQU24_RS22145,EQU24_RS22150,EQU24_RS22155
0,MeOH,23.333155,18.915775,18.453916,18.267805,16.960643,12.377795,43.815536,9.670950,7.302145,...,1298.257682,15.624619,20.208066,26.004364,20.960234,28.719983,93.616437,161.528124,496.990651,280.344047
1,NO3_lowO2_slow_growth,32.050358,43.656760,21.351623,62.267687,41.684925,31.921455,57.849768,16.885694,14.926147,...,6497.868109,26.273485,28.945133,23.525245,26.432667,35.167264,178.996199,164.083806,433.438735,493.895115
2,NoCu,44.348687,59.629360,28.268717,56.818319,49.839406,38.394652,81.530362,40.501969,36.576500,...,8345.785345,43.065124,34.380565,44.419579,34.601933,65.339879,253.608495,273.284694,731.052190,1087.621126
3,NoLanthanum,33.444023,43.689839,23.172675,57.297047,42.367072,41.941657,102.513601,30.226787,19.462312,...,5085.637409,16.423284,35.588138,44.623117,43.201743,21.927260,109.783330,67.277718,211.575175,328.943746
4,WithLanthanum,35.462185,41.792237,20.644554,57.130166,34.258335,46.201637,110.721781,31.813805,19.438086,...,3942.957792,15.972203,34.318829,49.216725,40.000662,21.220809,98.100610,73.116973,194.389586,319.998959


In [9]:
top_10_genes = get_top_n_perc_by_condition(df_means, LOCI, 10)
top_5_genes = get_top_n_perc_by_condition(df_means, LOCI, 5)
top_3_genes = get_top_n_perc_by_condition(df_means, LOCI, 3)
top_1_genes = get_top_n_perc_by_condition(df_means, LOCI, 1)


In [10]:
def maine():
    gb_file = "data/5GB1c_sequence.gb"
    feats = get_features_from_genbank(gb_file,override=True)
    feats_filt = [x for x in feats if x[TYPE_IDX] != 'gene']
        
    # separate pos and neg lists and sort by gene start coordinate
    pos_feats = [x for x in feats_filt if x[STRAND_IDX]== 1]
    pos_feats.sort(key=lambda x: x[LEFT_IDX])
    
    neg_feats = [x for x in feats_filt if x[STRAND_IDX]==-1]
    neg_feats.sort(key=lambda x: x[RIGHT_IDX])
    
    # make sure we still keep all features
    assert(len(feats_filt) == len(pos_feats)+len(neg_feats))
    
    print("All feats:", len(feats))
    print("Filt feats:", len(feats_filt))
    
    # flag loci potentially in operons (set of locus ids)
    maybe_operon_loci = flag_potential_operon_loci(pos_feats, neg_feats, 120)
    
    for i in range(1,11):
        # get top locus ids in all conditions
        top_locs = get_top_n_perc_by_condition(df_means, LOCI, i)
        
        # filter out loci maybe in operons
        top_locs_op_filter_out = [x for x in top_locs if x in maybe_operon_loci]
        print(f'Top {i}%: {len(top_locs)} ({len(top_locs) - len(top_locs_op_filter_out)} filtered)')

        # save to a two-column txt file
        outf = f'output/loci_in_top_{i}perc.txt'
        write_loci_to_file(top_locs,top_locs_op_filter_out, outf)    

In [11]:
maine()

All feats: 8862
Filt feats: 4431
Top 1%: 14 (8 filtered)
Top 2%: 26 (17 filtered)
Top 3%: 37 (25 filtered)
Top 4%: 56 (35 filtered)
Top 5%: 84 (43 filtered)
Top 6%: 107 (55 filtered)
Top 7%: 127 (65 filtered)
Top 8%: 153 (78 filtered)
Top 9%: 178 (92 filtered)
Top 10%: 197 (103 filtered)
